In [74]:
import pandas as pd 
import numpy as np
#tdqm = progress bar
from tqdm import tqdm
from datetime import datetime

from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn import utils



In [75]:
# Number of rows
nrows = 100000
# Chunksize = 10000 
chunksize = 10000
# Vitou's path
# path = 'C:/Users/sirus/Downloads/train.csv'
# Muna's path
path = '/Users/muna/Development/DataScience/new-york-city-taxi-fare-prediction/train.csv'
# Columns to read from the data
# Todo :split date into meaningful data
# cols = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','fare_amount']
cols = ['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','fare_amount']

#list to hold the batch dataframe
df_list = []

for df_chunk in tqdm(pd.read_csv(path,usecols=cols, chunksize=chunksize,nrows=nrows)):
    df_list.append(df_chunk)
    
# Merge all dataframes into one dataframe
data = pd.DataFrame()
data = pd.concat(df_list)
# # Delete the dataframe list to release memory
del df_list, df_chunk

10it [00:00, 43.31it/s]


In [76]:
# Given a dataframe, add two new features 'abs_diff_longitude' and
# 'abs_diff_latitude' reprensenting the "Manhattan vector" from
# the pickup location to the dropoff location.
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(data)

In [77]:
%%time
print(data[::10])

       fare_amount          pickup_datetime  pickup_longitude  \
0              4.5  2009-06-15 17:26:21 UTC        -73.844311   
10             5.3  2012-04-08 07:30:50 UTC        -73.996335   
20            10.5  2010-09-07 13:18:00 UTC        -73.985382   
30             5.5  2014-02-19 16:03:00 UTC        -73.976075   
40             8.1  2011-03-29 15:39:49 UTC        -73.984267   
...            ...                      ...               ...   
99950         31.0  2013-04-27 15:19:33 UTC        -73.997256   
99960          7.3  2011-09-07 06:39:00 UTC        -73.954227   
99970          5.7  2010-06-30 21:37:56 UTC        -73.968700   
99980         14.1  2011-03-21 15:05:00 UTC        -73.984670   
99990         13.0  2014-09-07 03:36:58 UTC        -73.991776   

       pickup_latitude  dropoff_longitude  dropoff_latitude  passenger_count  \
0            40.721319         -73.841610         40.712278                1   
10           40.737142         -73.980721         40.733559

### Let's take a look at the data

In [78]:
%%time
data.describe(include='all')

CPU times: user 209 ms, sys: 5.97 ms, total: 215 ms
Wall time: 218 ms


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,abs_diff_longitude,abs_diff_latitude
count,100000.000000,100000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
unique,NaN,98245,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2013-12-09 19:25:00 UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,11.354652,NaN,-72.494682,39.914481,-72.490967,39.919053,1.673820,0.160038,0.096688
std,9.716777,NaN,10.693934,6.225686,10.471386,6.213427,1.300171,3.742760,2.322124
min,-44.900000,NaN,-736.550000,-74.007670,-84.654241,-74.006377,0.000000,0.000000,0.000000
25%,6.000000,NaN,-73.992041,40.734996,-73.991215,40.734182,1.000000,0.005767,0.006582
50%,8.500000,NaN,-73.981789,40.752765,-73.980000,40.753243,1.000000,0.012485,0.013867
75%,12.500000,NaN,-73.966982,40.767258,-73.963433,40.768166,2.000000,0.023911,0.026925


###  Check for missing values


In [79]:
%%time
# Checking for missing values
data.isnull().sum()

CPU times: user 25.2 ms, sys: 9.59 ms, total: 34.8 ms
Wall time: 23.9 ms


fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
passenger_count       0
abs_diff_longitude    0
abs_diff_latitude     0
dtype: int64

### From our observation, there are some missing values, so we replace it with the mean of that column

In [80]:
%%time
data = data.dropna(how = 'any', axis = 'rows')

CPU times: user 22.5 ms, sys: 4.22 ms, total: 26.7 ms
Wall time: 24.9 ms


###  From the description above, some of the fare_amount have negative values

#### We are going to remove all negatvie fare_amounts

In [81]:
%%time
data = data[(data['fare_amount'] > 0)]


CPU times: user 8.44 ms, sys: 2.16 ms, total: 10.6 ms
Wall time: 11.4 ms


In [82]:
%%time
data.describe()

CPU times: user 70.8 ms, sys: 3.18 ms, total: 74 ms
Wall time: 88.1 ms


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,abs_diff_longitude,abs_diff_latitude
count,99988.000000,99988.000000,99988.000000,99988.000000,99988.000000,99988.000000,99988.000000,99988.000000
mean,11.356728,-72.494509,39.914382,-72.491533,39.919360,1.673781,0.159317,0.096292
std,9.714644,10.694564,6.226052,10.469494,6.212511,1.300098,3.735735,2.318719
min,0.010000,-736.550000,-74.007670,-84.654241,-74.006377,0.000000,0.000000,0.000000
25%,6.000000,-73.992041,40.734996,-73.991215,40.734182,1.000000,0.005768,0.006584
50%,8.500000,-73.981789,40.752765,-73.980000,40.753243,1.000000,0.012485,0.013869
75%,12.500000,-73.966986,40.767258,-73.963438,40.768164,2.000000,0.023911,0.026925
max,200.000000,40.787575,401.083332,40.851027,404.616667,6.000000,662.561258,363.934787


### In further observation, we have pickup_longitude, puckup_latitiude, dropoff_longitude and dropoff_latitude. We can calculate the distance

In [83]:
# Function that calculates distance between pickup location and dropoff location
def getDistance(lat1,lon1,lat2,lon2):
    r = 6378 # earth's radius
    lat1 = np.deg2rad(lat1)
    lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2)
    lon2 = np.deg2rad(lon2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = r*c
    
    return distance

In [84]:
%%time
# Add new cloumn "distance" to the data
data['distance'] = getDistance(data.pickup_latitude, data.pickup_longitude, 
                                      data.dropoff_latitude, data.dropoff_longitude)

CPU times: user 40.7 ms, sys: 5.48 ms, total: 46.2 ms
Wall time: 24.9 ms


In [85]:
data.describe(include='all')

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,abs_diff_longitude,abs_diff_latitude,distance
count,99988.000000,99988,99988.000000,99988.000000,99988.000000,99988.000000,99988.000000,99988.000000,99988.000000,99988.000000
unique,NaN,98233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2009-06-29 09:12:00 UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,11.356728,NaN,-72.494509,39.914382,-72.491533,39.919360,1.673781,0.159317,0.096292,18.528075
std,9.714644,NaN,10.694564,6.226052,10.469494,6.212511,1.300098,3.735735,2.318719,356.128990
min,0.010000,NaN,-736.550000,-74.007670,-84.654241,-74.006377,0.000000,0.000000,0.000000,0.000000
25%,6.000000,NaN,-73.992041,40.734996,-73.991215,40.734182,1.000000,0.005768,0.006584,1.215962
50%,8.500000,NaN,-73.981789,40.752765,-73.980000,40.753243,1.000000,0.012485,0.013869,2.121788
75%,12.500000,NaN,-73.966986,40.767258,-73.963438,40.768164,2.000000,0.023911,0.026925,3.892159


In [ ]:
%%time
data.head()

In [ ]:
feature_cols = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','abs_diff_longitude',
                'abs_diff_latitude','distance','year','month','day','hour']

X = data[feature_cols] 
y = data['fare_amount']

In [ ]:
X = preprocessing.scale(X)
# i don't think we should scale y (target). Professor Mo said to only scale features columns
# y = preprocessing.scale(y)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                 random_state=2)
#55millions rows it = iteration(nrows per it) x nrows = total rows

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
#linear regression can also use y_train
my_linreg = LinearRegression()
my_linreg.fit(X_train, y_train)
y_prediction = my_linreg.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_prediction)
rmse = np.sqrt(mse)
print(rmse)

In [ ]:
my_logreg = LogisticRegression()
my_logreg.fit(X_train, y_train)
y_prediction = my_logreg.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_prediction)
rmse = np.sqrt(mse)
print(rmse)

In [ ]:
%%time
my_decisiontree = DecisionTreeClassifier(random_state=5)
my_decisiontree.fit(X_train, y_train)
y_prediction = my_decisiontree.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_prediction)
rmse = np.sqrt(mse)
print(rmse)

Cross Validation

In [ ]:
%%time
score_list = cross_val_score(my_linreg, X, y_encoded, cv=10)
score = score_list.mean()
print('KNN: '+str(score))

In [ ]:
%%time
score_list = cross_val_score(my_logreg, X, y_encoded, cv=10)
score = score_list.mean()
print('Logistic Regression: '+str(score))

In [ ]:
%%time
score_list = cross_val_score(my_decisiontree, X, y_encoded, cv=10)
score = score_list.mean()
print('Decision Tree: '+str(score))